# Design
###### Created by [Roberto Mora](https://github.com/romogo17)

I will be structuring the `pgcv` datatypes and functions in different PostgreSQL schemas. This schemas will group the operations logically. The structure is somewhat base on OpenCV.

The schemas will be:
1. `pgcv_core`
1. `pgcv_io`
1. `pgcv_filter`
1. `pgcv_histogram`
1. `pgcv_feature`
1. `pgcv_draw`
1. `pgcv_shapes`




## `pgcv_core`

This schema contains the datatypes used in the `pgcv` extension. These datatypes are basically a representation of a N-dimension array in PostgreSQL. The idea is to map the `numpy.ndarray` to theese structures.

There are 4 types of `ndarray`s. The `int`, `numeric` and `varchar` correspond to the built-int PostgreSQL datatypes.

The `descriptor` is used to describe the columns of a two dimentional array (matrix) or a set of one dimentional array (vector)

```SQL
CREATE TYPE pgcv_core.ndarray_int4 AS (
    shape   int[],
    data    int[]
);

CREATE TYPE pgcv_core.ndarray_numeric AS (
    shape   int[],
    data    numeric[]
);

CREATE TYPE pgcv_core.ndarray_varchar AS (
    shape   int[],
    data    varchar[]
);

CREATE TYPE pgcv_core.descriptor AS (
    elems   int,
    data    varchar[]
);

```

On the other hand, there are some functions that were incorporated into the core of the extension. For instance `hash_avg` calculates the average hash of an image

```SQL
CREATE OR REPLACE FUNCTION pgcv_core.hash_avg(image pgcv_core.ndarray_int4, size int DEFAULT 8)
  RETURNS varchar
```

This function can be called with the following code

```SQL
-- having an image in the database (size defaults to 8)
SELECT pgcv_core.hash_avg(<image>, [<size>]);
```


## `pgcv_io`

This schema contains the image input and output functions to the filesystem. Meaning that this functions read and write images into files.

```SQL
CREATE OR REPLACE FUNCTION pgcv_io.image_read(filename varchar)
  RETURNS pgcv_core.ndarray_int4

CREATE OR REPLACE FUNCTION pgcv_io.image_write(image pgcv_core.ndarray_int4, filename varchar)
  RETURNS boolean
```

These functions can be tested with the following code

```SQL
-- having a filename that represents an image
SELECT shape FROM pgcv_io.image_read('<filename>');

-- having an image in the database and the output filename
SELECT pgcv_io.image_write(<image>, '<filename>');
```

## `pgcv_filtering`

This schema contains the image filtering functions. One example of this functions is the `median_blur` which replaces each pixel by the median of a local window array given by a kernel size.

```SQL
CREATE OR REPLACE FUNCTION pgcv_filter.blur_median(image pgcv_core.ndarray_int4, kernel int DEFAULT 3)
  RETURNS pgcv_core.ndarray_int4
  
CREATE OR REPLACE FUNCTION pgcv_filter.threshold_otsu(image pgcv_core.ndarray_int4)
  RETURNS float
  
CREATE OR REPLACE FUNCTION pgcv_filter.enhancement_otsu(image pgcv_core.ndarray_int4)
  RETURNS pgcv_core.ndarray_int4
```

These functions can be tested with the following code

```SQL
-- having an image in the database and odd kernel size (kernel size defaults to 3 if not specified)
SELECT pgcv_filter.median_blur(<image>, [<kernel size>]);

-- having an image in the database
SELECT pgcv_filter.threshold_otsu(<image>);

-- having an image in the database
SELECT pgcv_filter.enhancement_otsu(<image>);
```


## `pgcv_histogram`

This schema contains the histogram computing functions. There are two main kinds of histograms in `pgcv`, both return an histogram and a set of bin features (either the center of the bins or the edges)

```SQL
CREATE OR REPLACE FUNCTION pgcv_histogram.hist_bin_edges(image pgcv_core.ndarray_int4, bins int DEFAULT 10, 
                                                         as_float boolean DEFAULT TRUE,
  OUT hist numeric[], OUT bin_edges numeric[])
  
CREATE OR REPLACE FUNCTION pgcv_histogram.hist_bin_centers(image pgcv_core.ndarray_int4, bins int DEFAULT 10,
  OUT hist numeric[], OUT bin_centers numeric[])
```

These functions can be tested with the following code

```SQL
-- having an image in the database, the number of bins (bins defaults to 10 if not specified) and whether the histogram has to be normalized or not
SELECT * FROM pgcv_histogram.hist_bin_edges(<image>, [<bins>, [<as_float>]]);

-- having an image in the database and the number of bins (bins defaults to 10 if not specified) 
SELECT * FROM pgcv_histogram.hist_bin_centers(<image>, [<bins>]);
```


In [34]:
import numpy as np
from PIL import Image
from scipy import signal
from skimage import filters, exposure, img_as_float

img = Image.open('/Users/ro/U/[ Asistencia ] - Proyecto de Investigacion/Source_Images/mdb155.pgm')
img = np.array(img).astype('uint8')

#hist, bin_edges = np.histogram(img, 5)

img2 = img_as_float(img)
hist2, bin_centers = exposure.histogram(img2, nbins=5)

#print(hist, bin_edges)
print(hist2, bin_centers)

0.26988829818426396
[567650  31754 106704 321948  20520] [0.08980392 0.26941176 0.44901961 0.62862745 0.80823529]
